In [1]:
from bs4 import BeautifulSoup
import requests
import os
from io import open

In [2]:
url = "https://www.elmundo.es/ciencia-y-salud/salud/2021/12/07/61aef71b3cd280001f293bf1-directo.html"


   
pagina = requests.get(url)
soup = BeautifulSoup(pagina.content, 'html.parser')


date_length = 10 # 2021-12-03T18:09:39.529Z


time=soup.find("time")


time.get('datetime')

'2021-12-08T12:48:37Z'

In [3]:
url="https://elpais.com/planeta-futuro/2021-12-06/las-muertes-por-malaria-aumentaron-hasta-las-627000-en-2020-por-la-crisis-de-la-covid-19.html"
pagina = requests.get(url)
soup = BeautifulSoup(pagina.content, 'html.parser')

date_length = 10 # 2021-12-03T18:09:39.529Z
soup.find("a", id="article_date_p")["data-date"][:date_length]

'2021-12-06'

In [4]:
url="https://www.elmundo.es/ciencia-y-salud/salud/2021/12/07/61af78ca21efa0fe778b45d0.html"

pagina = requests.get(url)
soup = BeautifulSoup(pagina.content, 'html.parser')
date_length = 10 # 2021-12-03T18:09:39.529Z
time.get('datetime')[:date_length]

'2021-12-08'

In [5]:
def elMundoPalabrasLlaves(soup):
    palabras_llaves = ""
    separador = ", "
    # listo con todos los palabras llaves
    html_llaves = soup.find("ul", class_="ue-c-article__tags ue-l-article--leftcol-width-from-desktop ue-l-article--float-left-from-desktop ue-l-article--move-to-leftcol-from-desktop ue-l-article--bottom-absolute-from-desktop")
    # solo los palabras que son enlaces
    enlaces_llaves = html_llaves.find_all("a")
    # sacar el texto del enlace y anadir al listo de todos
    for enlace in enlaces_llaves:
        palabras_llaves = palabras_llaves + enlace.text + separador
    # quitar ultima seperador
    palabras_llaves = palabras_llaves[:-(len(separador))]
    return palabras_llaves

In [6]:
def elMundoTexto(soup):
    content = ""
    # div con todo el texto
    text = soup.find("div", class_="ue-l-article__body ue-c-article__body")
    # buscar todos los p
    paragraphs = text.find_all("p")
    # sacar el texto del p y anadir al content
    for paragraph in paragraphs:
        content = content + paragraph.text + "\n"
    # print(content)
    return content

In [7]:
def elMundoTitulo(soup):
    return soup.find("h1", class_="ue-c-article__headline js-headline").text

In [8]:
def elMundoEntradilla(soup):
    return soup.find("p", class_="ue-c-article__standfirst").text

In [9]:
def elMundoFecha(soup):
    date_length = 10 # 2021-12-03T18:09:39.529Z
    time=soup.find("time")
    return time.get('datetime')[:date_length]

In [14]:
def guardarFichero(contenido, fecha, diario_texto, categoria):
# calcular numero (nnn)
    
    counter = 1
    if fecha in counter_dictionary[categoria]:
        counter = counter_dictionary[categoria][fecha]
        counter_dictionary[categoria][fecha] += 1
    else:
        counter_dictionary[categoria][fecha] = 2

    # /diario/categoria/categoria.fecha.nnn.txt
  
    abs_path = os.getcwd()
    rel_path = diario_texto + "/" + categoria + "/"
    nombre_fichero = diario_texto + "." + categoria + "." + fecha + "." + str(counter).zfill(3) + ".txt"
    path = os.path.join(abs_path, rel_path, nombre_fichero)
    
    try:
        with open(path,'x') as fichero:
            fichero.write(contenido)
            print ("File " + nombre_fichero + " created!")
    except IOError:
        
        print("File " + nombre_fichero + " already exists...")
        

In [11]:
def elMundoWebScraping(pagina, diario_texto, categoria):
    soup = BeautifulSoup(pagina, 'html.parser')

    palabras_llaves = elMundoPalabrasLlaves(soup)
    titulo = elMundoTitulo(soup)
    entradilla = elMundoEntradilla(soup)
    texto = elMundoTexto(soup)
    fecha = elMundoFecha(soup)

    content = palabras_llaves + "\n\n" + titulo + "\n\n" + entradilla + "\n\n" + texto
    guardarFichero(content,fecha,diario_texto, categoria)
    return content

In [12]:
def elMundo(diario_texto, categoria, url):
    counter_dictionary[categoria] = {}
    pagina = requests.get(url)
    soup = BeautifulSoup(pagina.content, 'html.parser')

    enlaces = []
    for article in soup.find_all("article"):
        enlaces.append(article.find("a")['href'])

    for enlace in enlaces:
        articulo = requests.get(enlace)
        try:
            elMundoWebScraping(articulo.content, diario_texto, categoria)
        except:
            print("Error procesando articulo, saltando el articulo " + enlace)

In [16]:
diario = "https://elmundo.es"
diario_texto = "elMundo"
counter_dictionary = {}



categoria = "salud"
url = "https://www.elmundo.es/ciencia-y-salud/salud.html" 
elMundo(diario_texto, categoria, url)

categoria = "ciencia"
url = "https://www.elmundo.es/ciencia-y-salud/ciencia.html"
elMundo(diario_texto, categoria, url)

categoria = "tecnologia"
url = "https://www.elmundo.es/tecnologia.html"
elMundo(diario_texto, categoria, url)

Error procesando articulo, saltando el articulo https://www.elmundo.es/ciencia-y-salud/salud/2021/12/08/61b0a017fc6c83a81a8b4598.html
File salud.2021-12-08.001.txt created!
File salud.2021-12-08.002.txt created!
File salud.2021-12-08.003.txt created!
File salud.2021-12-07.001.txt created!
File salud.2021-12-07.002.txt created!
File salud.2021-12-07.003.txt created!
Error procesando articulo, saltando el articulo https://www.elmundo.es/ciencia-y-salud/salud/2021/12/07/61af53dbfdddff8e368b456d.html
File salud.2021-12-07.004.txt created!
File salud.2021-12-07.005.txt created!
File salud.2021-12-07.006.txt created!
File salud.2021-12-07.007.txt created!
Error procesando articulo, saltando el articulo https://www.elmundo.es/ciencia-y-salud/salud/2021/12/06/61ae4693fc6c8335578b4580.html
File salud.2021-12-06.001.txt created!
File salud.2021-12-06.002.txt created!
File salud.2021-12-06.003.txt created!
File salud.2021-12-06.004.txt created!
File salud.2021-12-05.001.txt created!
File salud.20